In [1]:
import os
import glob
import random
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_recall_fscore_support, accuracy_score, f1_score, matthews_corrcoef, confusion_matrix
import warnings 
warnings.filterwarnings("ignore")
%matplotlib inline
import tensorflow as tf
from tensorflow.keras import layers, Model
from keras.callbacks import EarlyStopping

In [2]:
import os
import numpy as np
import pandas as pd
import cv2

# Function to create directory if it does not exist
def create_directory(path):
    if not os.path.exists(path):
        os.makedirs(path)

# Base path to the synthetic dataset
base_path = '/kaggle/input/movement-dataset-entire/synthetic_dataset'
# Path to save the frames
frames_base_dir = '/kaggle/working/frames'

test_dir ='/kaggle/input/entire-test-movement-dataset/Abhishek_prashant_input_data'
frame_test_dir ='/kaggle/working/test/frames'

# Create the directory to save frames
create_directory(frames_base_dir)
create_directory(frame_test_dir)
# Initialize lists to hold the dataset

In [3]:
import os
import cv2
import numpy as np
import pandas as pd

# Initialize lists to store paths and labels
train_image_paths = []
train_foot_contacts = []

# Set a limit on the number of frames to process
frame_limit = 6000
total_frames = 0
stop_processing = False

# Loop through each person folder
for person_folder in os.listdir(base_path):
    if stop_processing:  # Check if processing should stop
        break

    person_path = os.path.join(base_path, person_folder)
    if os.path.isdir(person_path):
        # Loop through each activity folder
        for activity_folder in os.listdir(person_path):
            if stop_processing:  # Check if processing should stop
                break

            activity_path = os.path.join(person_path, activity_folder)
            if os.path.isdir(activity_path):
                # Path to the .npy file containing foot contact data
                foot_contacts_path = os.path.join(activity_path, 'foot_contacts.npy')
                
                if os.path.exists(foot_contacts_path):
                    # Load foot contact labels
                    foot_contact_labels = np.load(foot_contacts_path)
                    
                    # Construct the pattern to match video files
                    video_pattern = f'{activity_folder}_view0.mp4'
                    
                    # Loop through each video file in the activity folder
                    for video_file in os.listdir(activity_path):
                        if stop_processing:  # Check if processing should stop
                            break

                        if video_file == video_pattern:
                            video_path = os.path.join(activity_path, video_file)
                            
                            # Directory to save the frames of the current video
                            frames_dir = os.path.join(frames_base_dir, f'{person_folder}_{activity_folder}')
                            os.makedirs(frames_dir, exist_ok=True)  # Create directory if it doesn't exist
                            
                            # Capture the video from the file
                            cap = cv2.VideoCapture(video_path)
                            if not cap.isOpened():
                                print(f"Error: Could not open video {video_path}")
                                continue
                            
                            frame_count = 0
                            while cap.isOpened():
                                ret, frame = cap.read()
                                if not ret:
                                    break
                                
                                # Save frame as image
                                frame_filename = f'frame_{frame_count:04d}.jpg'
                                frame_path = os.path.join(frames_dir, frame_filename)
                                cv2.imwrite(frame_path, frame)
                                
                                # Append frame path and corresponding foot contact label to lists
                                if frame_count < len(foot_contact_labels):
                                    train_image_paths.append(frame_path)
                                    train_foot_contacts.append(foot_contact_labels[frame_count])
                                
                                frame_count += 1
                                total_frames += 1
                                
                                # Check if the frame limit is reached
                                if total_frames >= frame_limit:
                                    stop_processing = True  # Set flag to stop further processing
                                    print("Frame limit reached.")
                                    break
                            
                            # Release the capture
                            cap.release()
                            print(f"Processed {frame_count} frames from {video_file}")

# Create DataFrame
data = {
    'filename': train_image_paths,
    'left_heel': [label[0] for label in train_foot_contacts],
    'left_toe': [label[1] for label in train_foot_contacts],
    'right_heel': [label[2] for label in train_foot_contacts],
    'right_toe': [label[3] for label in train_foot_contacts]
}

left_leg_values = [label[0] & label[1] for label in train_foot_contacts]
right_leg_values = [label[2] & label[3] for label in train_foot_contacts]

data_left_leg = {
    'filename': train_image_paths,
    'left_leg': left_leg_values
}

data_right_leg = {
    'filename': train_image_paths,
    'right_leg': right_leg_values
}

train_left_df = pd.DataFrame(data_left_leg)
train_right_df = pd.DataFrame(data_right_leg)
train_df = pd.DataFrame(data)

print(train_df.head())
print(train_right_df.head())
print(train_left_df.head())
print(len(train_df))


Processed 60 frames from 449_swing_dancing_view0.mp4
Processed 60 frames from 1847_baseball_walk_in_view0.mp4
Processed 60 frames from 378_salsa_dancing_view0.mp4
Processed 60 frames from 321_boxing_view0.mp4
Processed 60 frames from 1854_baseball_step_up_to_bat_view0.mp4
Processed 60 frames from 43_walking_view0.mp4
Processed 60 frames from 189_baseball_pitching_view0.mp4
Processed 60 frames from 391_salsa_dancing_view0.mp4
Processed 60 frames from 2166_standing_turn_90_right_view0.mp4
Processed 60 frames from 376_salsa_dancing_view0.mp4
Processed 60 frames from 1855_baseball_milling_idle_view0.mp4
Processed 60 frames from 2165_standing_turn_90_left_view0.mp4
Processed 60 frames from 448_salsa_dancing_view0.mp4
Processed 60 frames from 392_salsa_dancing_view0.mp4
Processed 60 frames from 326_samba_dancing_view0.mp4
Processed 60 frames from 1851_baseball_hit_view0.mp4
Processed 60 frames from 1529_quarterback_pass_view0.mp4
Processed 60 frames from 328_boxing_view0.mp4
Processed 60 fra

In [4]:
import os
import cv2
import pandas as pd
test_image_paths = []
test_foot_contacts = []
# Function to create a directory if it doesn't exist
def create_directory(path):
    if not os.path.exists(path):
        os.makedirs(path)

# Directories
test_dir = '/kaggle/input/entire-test-movement-dataset'
frame_test_dir = '/kaggle/working/test/frames'
create_directory(frame_test_dir)

# Paths to MP4 files and corresponding ground truth CSV files
video_paths = [
    '/kaggle/input/entire-test-movement-dataset/Abhishek_prashant_input_data/Abhishek/jumping_abhishek.mp4',
    #'/kaggle/input/entire-test-movement-dataset/Abhishek_prashant_input_data/Prashant/jumping_video_prashant_gupta.mp4'
]

ground_truth_paths = [
    '/kaggle/input/entire-test-movement-dataset/Abhishek_prashant_input_data/Abhishek/ground_truth/abhishek jumping video.csv',
    #'/kaggle/input/entire-test-movement-dataset/Abhishek_prashant_input_data/Prashant/ground_truth/prashant jumping video.csv'
]

# Initialize lists for DataFrame
test_image_paths = []
test_foot_contacts = []

# Set a limit on the number of frames to process
MAX_FRAMES = 200
total_frames_processed = 0

# Process each video and its corresponding ground truth
for video_path, gt_path in zip(video_paths, ground_truth_paths):
    if total_frames_processed >= MAX_FRAMES:
        break

    # Read the ground truth CSV
    ground_truth_df = pd.read_csv(gt_path)
    
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        continue
    
    # Extract video identifier (base name of video file without extension)
    video_id = os.path.splitext(os.path.basename(video_path))[0]
    
    frame_count = 0
    while cap.isOpened():
        if total_frames_processed >= MAX_FRAMES:
            break

        ret, frame = cap.read()
        if not ret:
            break
        
        # Save frame as image with video identifier
        frame_filename = f'{video_id}_frame_{total_frames_processed:04d}.jpg'
        frame_path = os.path.join(frame_test_dir, frame_filename)
        cv2.imwrite(frame_path, frame)
        
        # Get foot contact data for the current frame
        if frame_count < len(ground_truth_df):
            left_foot = ground_truth_df.iloc[frame_count]['left_foot']
            right_foot = ground_truth_df.iloc[frame_count]['right_foot']
            test_image_paths.append(frame_path)
            test_foot_contacts.append([left_foot, right_foot])
        
        frame_count += 1
        total_frames_processed += 1

    cap.release()
    print(f"Processed {frame_count} frames from {video_path}")

    if total_frames_processed >= MAX_FRAMES:
        break

# Create DataFrames
test_df = pd.DataFrame({
    'filename': test_image_paths,
    'left_leg': [label[0] for label in test_foot_contacts],
    'right_leg': [label[1] for label in test_foot_contacts]
})

# Create DataFrames for left and right foot separately if needed
test_left_df = pd.DataFrame({
    'filename': test_image_paths,
    'left_leg': [label[0] for label in test_foot_contacts]
})

test_right_df = pd.DataFrame({
    'filename': test_image_paths,
    'right_leg': [label[1] for label in test_foot_contacts]
})

# Print the first few rows of each DataFrame
print(test_df.head())
print(test_left_df.head())
print(test_right_df.head())


Processed 200 frames from /kaggle/input/entire-test-movement-dataset/Abhishek_prashant_input_data/Abhishek/jumping_abhishek.mp4
                                            filename  left_leg  right_leg
0  /kaggle/working/test/frames/jumping_abhishek_f...         1          1
1  /kaggle/working/test/frames/jumping_abhishek_f...         1          1
2  /kaggle/working/test/frames/jumping_abhishek_f...         1          1
3  /kaggle/working/test/frames/jumping_abhishek_f...         1          1
4  /kaggle/working/test/frames/jumping_abhishek_f...         1          1
                                            filename  left_leg
0  /kaggle/working/test/frames/jumping_abhishek_f...         1
1  /kaggle/working/test/frames/jumping_abhishek_f...         1
2  /kaggle/working/test/frames/jumping_abhishek_f...         1
3  /kaggle/working/test/frames/jumping_abhishek_f...         1
4  /kaggle/working/test/frames/jumping_abhishek_f...         1
                                            filena

In [5]:
# Function to load and preprocess image
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    try:
        img = cv2.imread(image_path.decode('utf-8'))
        if img is None:
            print(f"Failed to load image at path: {image_path}")
            return np.zeros((target_size[0], target_size[1], 3), dtype=np.float32)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        h, w, _ = img.shape
        if h > w:
            pad_width = (h - w) // 2
            padding = ((0, 0), (pad_width, h - w - pad_width), (0, 0))
        else:
            pad_height = (w - h) // 2
            padding = ((pad_height, w - h - pad_height), (0, 0), (0, 0))
        
        img = np.pad(img, padding, mode='constant', constant_values=255)
        img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)
        img = img / 255.0
        return img.astype(np.float32)
    except Exception as e:
        print(f"Error processing image at path: {image_path}, error: {e}")
        return np.zeros((target_size[0], target_size[1], 3), dtype=np.float32)

# Data Augmentation
data_augmentation = tf.keras.Sequential(
    [
        layers.RandomFlip("horizontal", input_shape=(224, 224, 3)),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ]
)

def create_dataset(image_paths, labels, batch_size, training=True):
    def load_and_preprocess_image_tf(image_path, label):
        img = tf.numpy_function(load_and_preprocess_image, [image_path], tf.float32)
        img.set_shape((224, 224, 3))
        return img, label

    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(load_and_preprocess_image_tf, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    if training:
        dataset = dataset.shuffle(buffer_size=len(image_paths))
        dataset = dataset.map(lambda x, y: (data_augmentation(tf.expand_dims(x, 0))[0], y), num_parallel_calls=tf.data.experimental.AUTOTUNE)
    
    dataset = dataset.batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset

# Load the base model

base_model = tf.keras.applications.ResNet50V2(  #VGG16
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3)
)

# Set the last 10% of layers to be trainable
total_layers = len(base_model.layers)
trainable_layers_start = int(total_layers * 0.67)
for layer in base_model.layers[:trainable_layers_start]:
    layer.trainable = False
for layer in base_model.layers[trainable_layers_start:]:
    layer.trainable = True

# Build the model
model = tf.keras.Sequential([
    base_model,
    layers.BatchNormalization(),
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model with Adam optimizer and custom learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.build((None, 224, 224, 3)) 
# Print the summary of the model
model.summary()

# Split data into training and validation sets
train_paths, val_paths, train_labels, val_labels = train_test_split(train_left_df['filename'].values, train_left_df['left_leg'].values, test_size=0.2, random_state=42)

# Prepare datasets
train_dataset = create_dataset(train_paths, train_labels, batch_size=4, training=True)  # Smaller batch size
val_dataset = create_dataset(val_paths, val_labels, batch_size=4, training=False)

# Train the model
steps_per_epoch = len(train_paths) // 4
validation_steps = len(val_paths) // 4

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',  # or 'val_loss'
    patience=5,  # number of epochs with no improvement after which training will be stopped
    verbose=1,
    restore_best_weights=True  # restores the model weights from the epoch with the best value of the monitored quantity
)

history = model.fit(train_dataset, validation_data=val_dataset, epochs=150, verbose=1,callbacks=[early_stopping])

94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50v2 (Functional)         │ (None, 7, 7, 2048)     │    23,564,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 7, 7, 2048)     │         8,192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,097,793 (91.93 MB)

 Trainable params: 17,999,873 (68.66 MB)

...

Epoch 1/150


I0000 00:00:1724700234.334137     468 service.cc:145] XLA service 0x7dc960003c30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1724700234.334205     468 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


   5/1200 ━━━━━━━━━━━━━━━━━━━━ 36s 30ms/step - accuracy: 0.5525 - loss: 0.6788

I0000 00:00:1724700245.812772     468 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1200/1200 ━━━━━━━━━━━━━━━━━━━━ 98s 37ms/step - accuracy: 0.6037 - loss: 0.6575 - val_accuracy: 0.6600 - val_loss: 0.5953
Epoch 2/150
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 73s 36ms/step - accuracy: 0.6504 - loss: 0.6164 - val_accuracy: 0.6483 - val_loss: 0.6050
Epoch 3/150
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 72s 36ms/step - accuracy: 0.6865 - loss: 0.5910 - val_accuracy: 0.7000 - val_loss: 0.5902
Epoch 4/150
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 69s 34ms/step - accuracy: 0.6976 - loss: 0.5751 - val_accuracy: 0.7133 - val_loss: 0.6215
Epoch 5/150
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 69s 34ms/step - accuracy: 0.7166 - loss: 0.5468 - val_accuracy: 0.7383 - val_loss: 0.5479
Epoch 6/150
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 68s 33ms/step - accuracy: 0.7350 - loss: 0.5205 - val_accuracy: 0.7308 - val_loss: 0.5856
Epoch 7/150
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 69s 34ms/step - accuracy: 0.7459 - loss: 0.4967 - val_accuracy: 0.7467 - val_loss: 0.6321
Epoch 8/150
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 70s 34ms/step - accuracy: 0.7816 - los